# Align WRF TSK with MODIS LST

This pipeline is for producing "aligned" dataset of WRF data (TSK variable) produced for the SERDP Fish and Fire project with 8-day MODIS LST data. Both datasets are re-projected to EPSG:3338 and aligned to the same grid (i.e. overlapping grid cells) and the maximum WRF TSK is re-sampled to match the same 8-day time periods used in the MODIS data. Future projections are resampled to the 8-day time periods estimated from assuming regular continuation of the observed MODIS 8-day scheme.

## 0 - Setup

Execute this setup cell before running any of the other steps in the notebook:

In [1]:
import os
from pathlib import Path

# paths provided for SNAP internal use
wrf_dir = Path(os.getenv("WRF_DIR") or "/import/SNAP/pbieniek/SERDP/hourly")
modis_dir = Path(os.getenv("MODIS_DIR") or f"{os.getenv('CENTER1')}/data/MODIS")
scratch_dir = Path(os.getenv("SCRATCH_DIR") or f"{os.getenv('CENTER1')}/data/modis_lst-scratch")
output_dir = Path(os.getenv("OUTPUT_DIR") or f"{os.getenv('HOME')}/projects/SERDP-fish-fire/data") 

## Split netCDFs into GeoTIFFs

This step is for splitting the netCDFs into GeoTIFFs, as this is the desired format for our project partners.

In [2]:
# setup cell
import numpy as np
import pandas as pd
import xarray as xr
import rasterio as rio
from rasterio.crs import CRS


modis_gtiff_dir = output_dir.joinpath("aligned-WRF-MODIS/MODIS/geotiff")
modis_gtiff_dir.mkdir(exist_ok=True)
wrf_gtiff_dir = output_dir.joinpath("aligned-WRF-MODIS/WRF/geotiff")
wrf_gtiff_dir.mkdir(exist_ok=True)

Define some functions to read a netCDF and output each slice as a geotiff to some directory:

In [3]:
def write_gtiff(arr, dst_fp, meta):
    with rio.open(dst_fp, "w", **meta) as dst:
        dst.write(arr, 1)
    return None


def netcdf_to_geotiff(nc_fp, gtiff_dir, out_base_fn):
    with xr.open_dataset(nc_fp) as ds:
        varname = [
            varname for varname in list(ds.variables) 
            if varname in ["lst", "tsk"]
        ][0]
        for date in ds.date.values:
            arr = ds[varname].sel(date=date).values
            pd_dt = pd.to_datetime(str(date)) 
            ymd = pd_dt.strftime('%Y%m%d')
            dst_fp = gtiff_dir.joinpath(out_base_fn.format(ymd))
            write_gtiff(arr, dst_fp, meta)

    print(f"{nc_fp} split into GeoTIFFs, written to {gtiff_dir}")
    return None

Setup the necessary metadata for writing the GeoTIFFs, it will be the same for all files:

In [4]:
with xr.open_dataset(output_dir.joinpath("aligned-WRF-MODIS/MODIS/lst_MOD11A2_aligned.nc")) as ds:
    height, width = [ds[dim].values.shape[0] for dim in ["yc", "xc"]]

# determine affine transform
# pixel resolution in epsg:3338
r = ds.xc.values[1] - ds.xc.values[0]
# divide by two for adding to get bounds
r2 = r / 2
transform = rio.transform.from_bounds(
    ds.xc.values[0] - r2,
    ds.yc.values[-1] - r2,
    ds.xc.values[-1] + r2,
    ds.yc.values[0] + r2,
    width,
    height,
)

meta = {
    "driver": "GTiff",
    "crs": CRS.from_epsg(3338),
    "compress": "lzw",
    "dtype": rio.float32,
    "count": 1,
    "height": height,
    "width": width,
    "transform": transform,
    "nodata": np.nan,
}

# template GeoTIFF filename
out_fn_template = "{}_{}_{}_{}_aligned.tif"

Iterate over MODIS files and break into GeoTIFFs:

In [8]:
modis_sensors = ["MOD11A2", "MYD11A2"]

for sensor in modis_sensors:
    nc_fp = output_dir.joinpath(f"aligned-WRF-MODIS/MODIS/lst_{sensor}_aligned.nc")
    out_base_fn = out_fn_template.format("modis", "lst", sensor.lower(), "{}")
    netcdf_to_geotiff(nc_fp, modis_gtiff_dir, out_base_fn)

/home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/MODIS/lst_MOD11A2_aligned.nc split into GeoTIFFs, written to /home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/MODIS/geotiff
/home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/MODIS/lst_MYD11A2_aligned.nc split into GeoTIFFs, written to /home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/MODIS/geotiff


And do the same for the WRF files:

In [5]:
wrf_models = ["era", "gfdl", "ccsm"]

for model in wrf_models:
    for nc_fp in output_dir.glob(f"aligned-WRF-MODIS/WRF/tsk_max_{model}_*_aligned.nc"):
        out_base_fn = out_fn_template.format("wrf", "max_tsk", model, "{}")
        netcdf_to_geotiff(nc_fp, wrf_gtiff_dir, out_base_fn)

/home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/WRF/tsk_max_era_2000-2018_aligned.nc split into GeoTIFFs, written to /home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/WRF/geotiff
/home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/WRF/tsk_max_gfdl_2008-2017_aligned.nc split into GeoTIFFs, written to /home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/WRF/geotiff
/home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/WRF/tsk_max_gfdl_2038-2047_aligned.nc split into GeoTIFFs, written to /home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/WRF/geotiff
/home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/WRF/tsk_max_gfdl_2068-2077_aligned.nc split into GeoTIFFs, written to /home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/WRF/geotiff
/home/kmredilla/projects/SERDP-fish-fire/data/aligned-WRF-MODIS/WRF/tsk_max_ccsm_2008-2017_aligned.nc split into GeoTIFFs, written to /home/kmredilla/projects/SE